<a href="https://colab.research.google.com/github/SaehyunC/localization/blob/master/answer/localization_assignment2_1D_Bayes_Rule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# sim

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import time


def create_poles(poles, distance):
    y = np.zeros(distance)
    for p in poles:
        y[p] = 1
    x = range(distance)
    plt.stem(x, y, use_line_collection=True)


def plot_poles(poles, distance):
    plt.subplot(311)
    plt.yticks([])
    plt.xticks([])
    plt.xlim([-0.9, distance + 0.9])
    create_poles(poles, distance)


def plot_belief(y, distance):
    plt.subplot(312)
    plt.yticks([])
    plt.xlim([-0.9, distance + 0.9])
    plt.plot([0, distance], [0, 0], '-b')
    x = range(distance)
    for i in range(distance):
        plt.plot([i, i], [0, y[i]], '-b')


def plot_current_measurement(loc, poles, distance):
    plt.subplot(313)
    plt.yticks([])
    plt.xticks([])
    plt.xlim([loc - 1.5, loc + 1.5])
    plt.ylim([-0.1, 1.1])

    plt.plot([loc + 0.2], [0.6], 'g<', markersize=40)
    plt.plot([loc], [0.4], 'bo', markersize=40)
    create_poles(poles, distance)


def plot(distance, poles, P_loc_i_posterior, robot, block=False, pause_time=1):
    if robot.detect_pole(poles):
        block = True
    plot_poles(poles, distance)
    plot_belief(P_loc_i_posterior, distance)
    plot_current_measurement(robot.loc, poles, distance)
    if block:
        plt.show()
    else:
        plt.pause(0.1)
        plt.show(block=block)
        plt.pause(pause_time)
        plt.close()

# assignment 2-1

In [ ]:
import numpy as np


class Robot:
    def __init__(self):
        self.loc = 0

    def move(self):
        self.loc += 1

    def detect_pole(self, poles):
        if self.loc + 1 in poles:
            return True
        else:
            return False


def bayes(PBA, PA, PB):
    # STUDENT CODE START
    # Make this function return the answer to Bayes Rule.

    # STUDENT CODE END
    return PAB


def shift_priors(P_loc_i):
    # STUDENT CODE START
    # Shift all probabilities to the right by one.

    # STUDENT CODE END


def update_loc_probability():
    # STUDENT CODE START
    # Perform Bayes Rule on each location.

    # STUDENT CODE END


distance = 40

robot = Robot()
robot.loc = 7
poles = [10, 15]

# P_loc_i_prior = P(Li) - Prior (before doing Bayes Rule) belief of being in location i.
# P_loc_i_posterior = P(Li|D) or P(Li|!D) - Updated belief of being in location i, after performing Bayes Rule.
# P_D = P(D) - Total Probability of Detecting a Pole.
# P_not_D = P(!D) - Total Probability of Not Detecting Pole.
# P_D_given_loc_i = P(D|Li) - Probability of detecting a pole, given we are in location i.
# P_not_D_given_loc_i = P(!D|Li) - Probability of not detecting a pole,
# given we are in location i.

# I'm initalizing variables (generally not done in python) so you can see what is a vector vs scalar.
# Setup bayes probabilities. Create as many probabilities or beliefs as
# there are discrete locations the robot can occupy.
P_loc_i_prior = np.zeros(distance)
P_loc_i_posterior = np.zeros(distance)
P_D = 0
P_not_D = 0
P_D_given_loc_i = np.zeros(distance)
P_not_D_given_loc_i = np.zeros(distance)

# STUDENT CODE START
# Set the prior as if the robot has equal probability to be in each location.

# Set probabilities of detecing a pole or not detecting a pole.

# Set the probabilities for detecting (or not detecting) a pole for each
# location i.

# STUDENT CODE END


# Setup done, run first calculation of robots location.
update_loc_probability()
plot(distance, poles, P_loc_i_posterior, robot, block=True)

# Begin Moving
for j in range(10):
    robot.move()
    # Shift the priors to follow the robots movement.
    shift_priors(P_loc_i_posterior)
    # Set prior to previous posterior, so we can start the cycle over again.
    P_loc_i_prior = P_loc_i_posterior
    # Perform Bayes Rule using new information about whether the robot can
    # detect a pole.
    update_loc_probability()
    plot(distance, poles, P_loc_i_posterior, robot)

plot(distance, poles, P_loc_i_posterior, robot, block=True, pause_time=1)

# assignment 2-2

In [ ]:
import numpy as np


class Robot:
    def __init__(self):
        self.loc = 0

    def move(self):
        self.loc += 1

    def detect_pole(self, poles):
        if self.loc + 1 in poles:
            return True
        else:
            return False


def bayes(PBA, PA, PB):
    # STUDENT CODE START
    # Make this function return the answer to Bayes Rule.

    # STUDENT CODE END


def shift_priors(P_loc_i):
    # STUDENT CODE START
    # Shift all probabilities to the right by one.

    # STUDENT CODE END


def update_loc_probability():
    # STUDENT CODE START
    # Perform Bayes Rule on each location.

    # STUDENT CODE END


distance = 40

robot = Robot()
robot.loc = 7
poles = [10, 15]

# P_loc_i_prior = P(Li) - Prior (before doing Bayes Rule) belief of being in location i.
# P_loc_i_posterior = P(Li|D) or P(Li|!D) - Updated belief of being in location i, after performing Bayes Rule.
# P_D = P(D) - Total Probability of Detecting a Pole.
# P_not_D = P(!D) - Total Probability of Not Detecting Pole.
# P_D_given_loc_i = P(D|Li) - Probability of detecting a pole, given we are in location i.
# P_not_D_given_loc_i = P(!D|Li) - Probability of not detecting a pole,
# given we are in location i.

# I'm initalizing variables (generally not done in python) so you can see what is a vector vs scalar.
# Setup bayes probabilities. Create as many probabilities or beliefs as
# there are discrete locations the robot can occupy.
P_loc_i_posterior = np.zeros(distance)
P_D_given_loc_i = np.zeros(distance)
P_D = 0
P_not_D = 0
P_not_D_given_loc_i = np.zeros(distance)
P_loc_i_prior = np.zeros(distance)

# STUDENT CODE START
# Set the prior as if the robot has equal probability to be in each location.

# Set probabilities of detecing a pole or not detecting a pole.

# Set the probabilities for detecting (or not detecting) a pole for each
# location i.

# STUDENT CODE END


# Setup done, run first calculation of robots location.
update_loc_probability()
plot(distance, poles, P_loc_i_posterior, robot, block=True)

# Begin Moving
for j in range(10):
    robot.move()
    # Shift the priors to follow the robots movement.
    shift_priors(P_loc_i_posterior)
    # Set prior to previous posterior, so we can start the cycle over again.
    P_loc_i_prior = P_loc_i_posterior
    # Perform Bayes Rule using new information about whether the robot can
    # detect a pole.
    update_loc_probability()
    plot(distance, poles, P_loc_i_posterior, robot)

plot(distance, poles, P_loc_i_posterior, robot, block=True, pause_time=1)